## The Big Data Challenge

# Import data

In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import requests


In [16]:
# Read in stringency data
file = (
    "/Users/amanda/Desktop/OneDrive - King's College London/Code First/Data_challenge/Big Data Challenge_ Data and Challenge Statements/Data/Stringency data/OxCGRT_latest_hackathon.csv"
)
stringency = pd.read_csv(file, parse_dates=['Date'])
print(stringency.head())

  CountryName CountryCode       Date  C1_School closing  C1_Flag  \
0       Aruba         ABW 2020-01-01                0.0      NaN   
1       Aruba         ABW 2020-01-02                0.0      NaN   
2       Aruba         ABW 2020-01-03                0.0      NaN   
3       Aruba         ABW 2020-01-04                0.0      NaN   
4       Aruba         ABW 2020-01-05                0.0      NaN   

   C2_Workplace closing  C2_Flag  C3_Cancel public events  C3_Flag  \
0                   0.0      NaN                      0.0      NaN   
1                   0.0      NaN                      0.0      NaN   
2                   0.0      NaN                      0.0      NaN   
3                   0.0      NaN                      0.0      NaN   
4                   0.0      NaN                      0.0      NaN   

   C4_Restrictions on gatherings  ...  StringencyIndex  \
0                            0.0  ...              0.0   
1                            0.0  ...              0.0

In [17]:
# Read in traffic data from ONS
file2 = (
    "/Users/amanda/Desktop/OneDrive - King's College London/Code First/Data_challenge/Traffic_ONS/10septembertrafficcamerasdataset.xlsx"
)
traffic = pd.read_excel(file2, sheet_name='Seasonally adjusted', header=[0, 1], skiprows=[2, 3], index_col=0)
print(traffic.head())

Daily counts London                                                      \
               Cars Motorbikes Buses Trucks Vans Pedestrians & cyclists   
2020-03-01        *          *     *      *    *                      *   
2020-03-02        *          *     *      *    *                      *   
2020-03-03        *          *     *      *    *                      *   
2020-03-04        *          *     *      *    *                      *   
2020-03-05        *          *     *      *    *                      *   

Daily counts North East                          ... Greater Manchester  \
                   Cars Motorbikes Buses Trucks  ...              Buses   
2020-03-01        55378         18  2249   1130  ...                  *   
2020-03-02        54773         24  2279   1159  ...                  *   
2020-03-03        56045         29  2283   1168  ...                  *   
2020-03-04        53209         20  2262   1150  ...                  *   
2020-03-05        52897 

In [12]:
# Read in the air quality data -- THIS DOESN'T WORK
url_air = ("http://api.erg.ic.ac.uk/AirQuality/Daily/MonitoringIndex/Latest/GroupName={GROUPNAME}/Json")
# Package the request, send the request and catch the response: r
air = requests.get(url_air)
# Decode the JSON data into a dictionary: json_data
json_data_air = air.json()

# # Print each key-value pair in json_data
# for k in json_data.keys():
#     print(k + ': ', json_data[k])


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Fill missing values

In [25]:
# Replace the missing values with None - THIS DOESN'T WORK
traffic = traffic.replace('\*', None)
print(traffic)

Daily counts  London                                                         \
                Cars Motorbikes  Buses Trucks   Vans Pedestrians & cyclists   
2020-03-01         *          *      *      *      *                      *   
2020-03-02         *          *      *      *      *                      *   
2020-03-03         *          *      *      *      *                      *   
2020-03-04         *          *      *      *      *                      *   
2020-03-05         *          *      *      *      *                      *   
...              ...        ...    ...    ...    ...                    ...   
2020-09-02    272143        548  17811  17905  39621                  41057   
2020-09-03    276908        598  18234  18855  40927                  45666   
2020-09-04    275792        594  18004  18385  40294                  46254   
2020-09-05    279997        564  17237  18026  39143                  42122   
2020-09-06    275020        568  16991  17475  38403

## Reformat data

In [18]:
# Melt traffic data
long_traffic = pd.melt(traffic, var_name=['City','Vehicle'], value_name='Count', ignore_index=False)
print(long_traffic)
long_traffic = pd.DataFrame(long_traffic.reset_index())

# Rename date column
long_traffic = long_traffic.rename(columns={"index": "Date"})
print(long_traffic)

               City                 Vehicle Count
2020-03-01   London                    Cars     *
2020-03-02   London                    Cars     *
2020-03-03   London                    Cars     *
2020-03-04   London                    Cars     *
2020-03-05   London                    Cars     *
...             ...                     ...   ...
2020-09-02  Reading  Pedestrians & cyclists    47
2020-09-03  Reading  Pedestrians & cyclists    48
2020-09-04  Reading  Pedestrians & cyclists    48
2020-09-05  Reading  Pedestrians & cyclists    41
2020-09-06  Reading  Pedestrians & cyclists    51

[7980 rows x 3 columns]
           Date     City                 Vehicle Count
0    2020-03-01   London                    Cars     *
1    2020-03-02   London                    Cars     *
2    2020-03-03   London                    Cars     *
3    2020-03-04   London                    Cars     *
4    2020-03-05   London                    Cars     *
...         ...      ...                     

In [19]:
# Create new columns with Year and Month
long_traffic.loc[:,'Year'] = long_traffic.loc[:,'Date'].dt.year
long_traffic.loc[:,'Month'] = long_traffic.loc[:,'Date'].dt.month
print(long_traffic.head())


long_traffic.loc[:,'Year'] = long_traffic.loc[:,'Year'].astype(int)
long_traffic.loc[:, 'Month'] = long_traffic.loc[:, 'Month'].astype(int)
print(long_traffic.info())

        Date    City Vehicle Count  Year  Month
0 2020-03-01  London    Cars     *  2020      3
1 2020-03-02  London    Cars     *  2020      3
2 2020-03-03  London    Cars     *  2020      3
3 2020-03-04  London    Cars     *  2020      3
4 2020-03-05  London    Cars     *  2020      3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7980 entries, 0 to 7979
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     7980 non-null   datetime64[ns]
 1   City     7980 non-null   object        
 2   Vehicle  7980 non-null   object        
 3   Count    7980 non-null   object        
 4   Year     7980 non-null   int64         
 5   Month    7980 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 374.2+ KB
None


In [21]:
long_traffic.columns

Index(['Date', 'City', 'Vehicle', 'Count', 'Year', 'Month'], dtype='object')

## Deal with missing values

In [9]:
long_traffic.loc[:, 'Count'] = long_traffic.loc[:, 'Count'].astype(str)
print(long_traffic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7980 entries, 0 to 7979
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     7980 non-null   datetime64[ns]
 1   City     7980 non-null   object        
 2   Vehicle  7980 non-null   object        
 3   Count    7980 non-null   object        
 4   Year     7980 non-null   int64         
 5   Month    7980 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 374.2+ KB
None


In [7]:
long_traffic.loc[:, 'Count'] = long_traffic.loc[:, 'Count'].str.replace('*', None)

TypeError: repl must be a string or callable

In [ ]:
## Check it can plot - just for de-bugging
sns.lineplot(
    data = long_traffic,
    x = 'Month',
    y = 'Count',
    aspect=3
)